In [98]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

In [81]:
def calculate_pagecount(total_items):
    if int(total_items)%100 ==0:
        pages = int(total_items)//100
    else:
        pages = (int(total_items)//100)+1
    return pages

## Boardgame Classification

In [92]:
ID = 306882

In [93]:
api_adress = f"https://www.boardgamegeek.com/xmlapi2/thing?id={ID}stats=1"

response = requests.get(api_adress)
response.status_code

200

In [22]:
xml = response.text

In [23]:
soup = BeautifulSoup(xml, 'xml')

In [51]:
#print(soup.prettify())

In [52]:
#soup.find_all('link')

In [39]:
categories = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamecategory'})]
categories

['Dice', 'Trains']

In [41]:
mechanics = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamemechanic'})]
mechanics

['Dice Rolling',
 'Line Drawing',
 'Network and Route Building',
 'Paper-and-Pencil',
 'Solo / Solitaire Game']

In [42]:
family = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamefamily'})]
family

['Admin: Unreleased Games',
 'Components: Dice with Icons',
 'Components: Dry Erase Markers / Dry Erase Surfaces',
 'Crowdfunding: Kickstarter',
 'Game: Railroad Ink',
 'Mechanism: Roll-and-Write']

In [46]:
expansions = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameexpansion'})]
expansions

['Railroad Ink Challenge: Epic Board',
 'Railroad Ink Challenge: Solo Board',
 'Railroad Ink: Arcade Expansion Pack',
 'Railroad Ink: Eldritch Expansion Pack',
 'Railroad Ink: Electricity Expansion Pack',
 'Railroad Ink: Engineer Expansion Pack',
 'Railroad Ink: Futuristic Expansion Pack',
 'Railroad Ink: Sky Expansion Pack',
 'Railroad Ink: Underground Expansion Pack']

In [47]:
integrations = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgameintegration'})]
integrations

['Railroad Ink Challenge: Lush Green Edition',
 'Railroad Ink: Blazing Red Edition',
 'Railroad Ink: Deep Blue Edition']

In [48]:
designers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamedesigner'})]
designers

['Hjalmar Hach', 'Lorenzo Silva']

In [49]:
publishers = [link.get('value') for link in soup.find_all('link', attrs={'type':'boardgamepublisher'})]
publishers 

['Horrible Guild', 'HeidelBÄR Games', 'White Goblin Games']

## User Ratings

In [59]:
api_adress = "https://www.boardgamegeek.com/xmlapi2/thing?id=306882&ratingcomments=1"

response = requests.get(api_adress)
response.status_code

200

In [60]:
xml = response.text
soup = BeautifulSoup(xml, 'xml')

In [110]:
#print(soup.prettify())

In [82]:
page = soup.find('comments').get('page')
page

'1'

In [83]:
total_items = soup.find('comments').get('totalitems')
total_items

'249'

In [85]:
pagecount = calculate_pagecount(total_items)
pagecount

3

In [91]:
ratings = []
users = []
comments = []
for comment in soup.find_all('comment'):
    ratings.append(comment.get('rating'))
    users.append(comment.get('username'))
    comments.append(comment.get('value'))
    

In [102]:
boardgame_id = [ID] * len(ratings)

In [103]:
ratings = pd.DataFrame({
    'id':boardgame_id,
    'user':users,
    'rating':ratings,
    'comment':comments
})

In [107]:
ratings.head(35)

,id,user,rating,comment
0,306882,basieellington,10,
1,306882,Zugli,10,
2,306882,Nyarly362,10,
3,306882,Serina24,10,
4,306882,Orbesteanu,10,
5,306882,Chouket76,10,
6,306882,ArBeMa,10,
7,306882,oliverivero,10,
8,306882,Norich,10,Nice new things added to the original game. Pl...
9,306882,lenlindsay,10,I'm getting the everything package (Collector).


In [105]:
ratings[ratings['user']=='Ser0']

,id,user,rating,comment
29,306882,Ser0,10,Still one of the best roll and writes!


In [109]:
ratings['rating'].isna().sum()

0

To DO:
- Loop through pages
- ignore users without ratings
- User IDs?